In [986]:
import raster_geometry as rg
import numpy as np
import matplotlib.pyplot as plt
from skimage.transform import rotate
import pickle
import sys
from skimage.transform import rescale, resize, downscale_local_mean
sys.path.insert(0,'/home/georgeos/Documents/GitHub/SYMPTOMM2')
import itertools
from joblib import Parallel, delayed
from skimage.morphology import opening
from PIL import Image       
import pymunk
from skimage.transform import PiecewiseAffineTransform, warp
from skimage import data
from copy import deepcopy
from tqdm import tqdm
import pandas as pd
from skimage import draw
import napari
from itertools import combinations
from SYMPTOMM import PSF
from matplotlib_scalebar.scalebar import ScaleBar
from cupyx.scipy.ndimage import convolve as cuconvolve
import tifffile
from skimage.exposure import match_histograms
import cupy as cp
from scipy.optimize import dual_annealing, shgo
from skimage.transform import resize
from skimage.metrics import structural_similarity as ssim
from scipy.optimize import basinhopping
import image_similarity_measures
from image_similarity_measures.quality_metrics import rmse, psnr, fsim, issm, sre, sam, uiq

In [987]:
def generate_curve_props(cell_timeseries):
    #Get unique cell IDs
    IDs = []
    for cell_list in cell_timeseries:
        for cell in cell_list:
            IDs.append(cell.ID)
    IDs = np.array(IDs)
    unique_IDs = np.unique(IDs)
    #For each cell, assign random curvature properties
    ID_props = []
    for ID in unique_IDs:
        freq_modif = (np.random.uniform(0.9,1.1)) # Choose one per cell
        amp_modif = (np.random.uniform(0.9,1.1)) # Choose one per cell
        phase_modif = np.random.uniform(-1,1)  # Choose one per cell
        ID_props.append([int(ID), freq_modif, amp_modif, phase_modif])
    ID_propps = np.array(ID_props)
    ID_propps[:,0] = ID_propps[:,0].astype(int)
    return np.array(ID_props)\

def gen_cell_props_for_draw(cell_timeseries_lists, ID_props):
    cell_properties = []
    for cell in cell_timeseries_lists:
        body, shape = (cell.body, cell.shape)
        vertices = []
        for v in shape.get_vertices():
            x,y = v.rotated(shape.body.angle) + shape.body.position #.rotated(self.shape.body.angle)
            vertices.append((x,y))
        vertices = np.array(vertices)

        centroid = get_centroid(vertices) 
        farthest_vertices = find_farthest_vertices(vertices)
        length = get_distance(farthest_vertices[0],farthest_vertices[1])
        width = cell.width
        #angle = np.arctan(vertices_slope(farthest_vertices[0], farthest_vertices[1]))
        angle = np.arctan2((farthest_vertices[0] - farthest_vertices[1])[1],(farthest_vertices[0] - farthest_vertices[1])[0])
        
        ID, freq_modif, amp_modif, phase_modif = ID_props[ID_props[:,0] == cell.ID][0]
        phase_mult = 20
        cell_properties.append([length, width, angle, centroid, freq_modif, amp_modif, phase_modif,phase_mult])
    return cell_properties

In [988]:
def raster_cell(length, width):
    radius = int(width/2)
    cyl_height = int(length - 2*radius)
    shape = 300 #200
    cylinder = rg.cylinder(
            shape = shape,
            height = cyl_height,
            radius = radius,
            axis=0,
            position=(0.5,0.5,0.5),
            smoothing=False)

    sphere1 = rg.sphere(shape,radius,((shape + cyl_height)/(2*shape),0.5,0.5))
    sphere2 = rg.sphere(shape,radius,((shape - cyl_height)/(2*shape),0.5,0.5))


    cell = (cylinder + sphere1 + sphere2)
    cell = cell[int(shape/2-cyl_height/2-radius-1):int(shape/2+cyl_height/2+radius+1),
                int(shape/2)-radius:int(shape/2)+radius,
               int(shape/2)-radius:int(shape/2)+radius]
    z,x,y = cell.nonzero()
    OPL_cell = np.sum(cell,axis=2)
    return OPL_cell

#OPL_cell = raster_cell(length = 55*2, width=15*2)
#plt.imshow(OPL_cell)
#plt.show()

def get_distance(vertex1, vertex2):
    return abs(np.sqrt((vertex1[0]-vertex2[0])**2 + (vertex1[1]-vertex2[1])**2))

def find_farthest_vertices(vertex_list):
    vertex_combs = list(itertools.combinations(vertex_list, 2))
    distance = 0
    farthest_vertices = 0
    for vertex_comb in vertex_combs:
        distance_ = get_distance(vertex_comb[0],vertex_comb[1])
        if distance_ > distance:
            distance = distance_
            farthest_vertices = vertex_comb
    return np.array(farthest_vertices)

def get_midpoint(vertex1, vertex2):
    x_mid = (vertex1[0]+vertex2[0])/2
    y_mid = (vertex1[1]+vertex2[1])/2
    return np.array([x_mid,y_mid])

def vertices_slope(vertex1, vertex2):
    return (vertex1[1] - vertex2[1])/(vertex1[0] - vertex2[0])

def midpoint_intercept(vertex1, vertex2):
    midpoint = get_midpoint(vertex1, vertex2)
    slope = vertices_slope(vertex1, vertex2)
    intercept = midpoint[1]-(slope*midpoint[0])
    return intercept

def get_centroid(vertices: list[tuple]) -> tuple:
    """Return the centroid of a list of vertices 
    
    Keyword arguments:
    vertices -- A list of tuples containing x,y coordinates.

    """
    return np.sum(vertices,axis=0)/len(vertices)

def place_cell(length, width, angle, position, space):
    angle = np.rad2deg(angle)
    x, y = np.array(position).astype(int)
    OPL_cell = raster_cell(length = length, width=width)
    rotated_OPL_cell = rotate(OPL_cell,angle,resize=True,clip=False,preserve_range=True)
    cell_y, cell_x = (np.array(rotated_OPL_cell.shape)/2).astype(int)
    offset_y = rotated_OPL_cell.shape[0] - space[y-cell_y:y+cell_y,x-cell_x:x+cell_x].shape[0]
    offset_x = rotated_OPL_cell.shape[1] - space[y-cell_y:y+cell_y,x-cell_x:x+cell_x].shape[1]
    space[y-cell_y+100:y+cell_y+offset_y+100,x-cell_x+100:x+cell_x+offset_x+100] += rotated_OPL_cell

In [989]:
with open("/home/georgeos/Storage/Dropbox (Cambridge University)/PhD_Georgeos_Hardo/ML_based_segmentation_results/40x_Ph2_test_1.5/output_pickles/cell_timeseries_short_trench.p", "rb") as f:
    cell_timeseries = pickle.load(f)
with open("/home/georgeos/Storage/Dropbox (Cambridge University)/PhD_Georgeos_Hardo/ML_based_segmentation_results/40x_Ph2_test_1.5/output_pickles/space_timeseries_short_trench.p", "rb") as f:
    space = pickle.load(f)


In [990]:
ID_props = generate_curve_props(cell_timeseries)
cell_timeseries_properties = Parallel(n_jobs=16)(delayed(gen_cell_props_for_draw)(a, ID_props) for a in cell_timeseries)

In [991]:
def transform_func(amp_modif, freq_modif, phase_modif):
    def perm_transform_func(x, amp_mult, freq_mult, phase_mult):
        return (amp_mult*amp_modif*np.cos((x/(freq_mult * freq_modif) - phase_mult * phase_modif)*np.pi)).astype(int)
    return perm_transform_func


In [992]:
do_transformation = False

def draw_scene(cell_properties):
    upscale = 1
    space_size = np.array([1000, 200]) * upscale
    space = np.zeros(space_size)
    space_masks = np.zeros(space_size)
    offsets = 50 * upscale
    for properties in cell_properties:
        length, width, angle, position, freq_modif, amp_modif, phase_modif,phase_mult = properties
        length = length*upscale; width = width * upscale; position = np.array(position) * upscale
        angle = np.rad2deg(angle) - 90
        x, y = np.array(position).astype(int) + offsets
        OPL_cell = raster_cell(length = length, width=width)
        
        if do_transformation:
            OPL_cell_2 = np.zeros((OPL_cell.shape[0],int(OPL_cell.shape[1]*2)))
            midpoint = int(np.median(range(OPL_cell_2.shape[1])))
            OPL_cell_2[:,midpoint-int(OPL_cell.shape[1]/2):midpoint-int(OPL_cell.shape[1]/2)+OPL_cell.shape[1]] = OPL_cell
            roll_coords = np.array(range(OPL_cell_2.shape[0]))
            freq_mult = (OPL_cell_2.shape[0])
            amp_mult = OPL_cell_2.shape[1]/10
            sin_transform_cell = transform_func(amp_modif, freq_modif, phase_modif)
            roll_amounts = sin_transform_cell(roll_coords,amp_mult,freq_mult,phase_mult)
            for B in roll_coords:
                OPL_cell_2[B,:] = np.roll(OPL_cell_2[B,:], roll_amounts[B])
            OPL_cell = (OPL_cell_2)
        
        rotated_OPL_cell = rotate(OPL_cell,angle,resize=True,clip=False,preserve_range=True)
        cell_y, cell_x = (np.array(rotated_OPL_cell.shape)/2).astype(int)
        offset_y = rotated_OPL_cell.shape[0] - space[y-cell_y:y+cell_y,x-cell_x:x+cell_x].shape[0]
        offset_x = rotated_OPL_cell.shape[1] - space[y-cell_y:y+cell_y,x-cell_x:x+cell_x].shape[1]
        space[
            y-cell_y:y+cell_y+offset_y  
              ,  x-cell_x  :  x+cell_x+offset_x  
             ] += rotated_OPL_cell
        space_masks[y-cell_y:y+cell_y+offset_y,x-cell_x:x+cell_x+offset_x] += (rotated_OPL_cell > 20)
        space_masks = space_masks == 1
        space_masks = opening(space_masks,np.ones((2,11)))
    return space, space_masks

In [ ]:
scenes = Parallel(n_jobs=15)(delayed(draw_scene)(cell_properties) for cell_properties in tqdm(cell_timeseries_properties))

 68%|██████▊   | 270/399 [01:58<00:59,  2.15it/s]

In [892]:
def scene_plotter(scene_array,output_dir,name,a,matplotlib_draw):
    if matplotlib_draw == True:
        plt.figure(figsize=(3,10))
        plt.imshow(scene_array)
        plt.tight_layout()
        plt.savefig(output_dir+"/{}_{}.png".format(name,str(a).zfill(3)))
        plt.clf()
        plt.close('all')
    else:
        im = Image.fromarray(scene_array.astype(np.uint8))
        im.save(output_dir+"/{}_{}.tif".format(name,str(a).zfill(3)))

In [10]:
output_dir = "/home/georgeos/Storage/Dropbox (Cambridge University)/PhD_Georgeos_Hardo/ML_based_segmentation_results/40x_Ph2_test_1.5/training_data/OPL_renders"
_ = Parallel(n_jobs=10)(delayed(scene_plotter)(scenes[x][1],output_dir,"OPL_mask",x+4000,matplotlib_draw=False) for x in range(len(scenes)))
_ = Parallel(n_jobs=10)(delayed(scene_plotter)(scenes[x][0],output_dir,"OPL_intensity",x+4000,matplotlib_draw=False) for x in range(len(scenes)))

# Phase Contrast

In [ ]:
##From here, prototyping phase contrast
def get_trench_segments(space):
    trench_shapes = []
    for shape, body in zip(space.shapes, space.bodies):
        if body.body_type == 2:
            trench_shapes.append(shape)
    return trench_shapes

trench_segment_props = []
for x in get_trench_segments(space):
    trench_segment_props.append([x.bb,x.area, x.a, x.b])
    
trench_segment_props = pd.DataFrame(trench_segment_props)
trench_segment_props.columns = ["bb", "area", "a", "b"]
main_segments = trench_segment_props.sort_values("area",ascending=False).iloc[0:2]

In [ ]:
main_segments = main_segments.reindex(index=main_segments.index[::-1]) # run if you get a zero-width image output


In [ ]:
offset = 50

trench_multiplier = 30
cell_multiplier = 2
background_multiplier = 10
grid_def = [range(15,trench_multiplier), np.arange(0.1,cell_multiplier,0.1), range(background_multiplier)]
grid = list(itertools.product(*grid_def))
def generate_PC_OPL(trench_multiplier,cell_multiplier,background_multiplier):
    segment_1_top_left = (0 + offset, int(main_segments.iloc[0]["bb"][0] + offset))
    segment_1_bottom_right = (int(main_segments.iloc[0]["bb"][3] + offset), int(main_segments.iloc[0]["bb"][2] + offset))

    segment_2_top_left = (0 + offset, int(main_segments.iloc[1]["bb"][0] + offset))
    segment_2_bottom_right = (int(main_segments.iloc[1]["bb"][3] + offset), int(main_segments.iloc[1]["bb"][2] + offset))

    test_scene = np.zeros(scenes[300][0].shape) + background_multiplier
    rr, cc = draw.rectangle(start = segment_1_top_left, end = segment_1_bottom_right, shape = test_scene.shape)
    test_scene[rr,cc] = 1 * trench_multiplier
    rr, cc = draw.rectangle(start = segment_2_top_left, end = segment_2_bottom_right, shape = test_scene.shape)
    test_scene[rr,cc] = 1 * trench_multiplier

    circ_midpoint_y = (segment_1_top_left[1] + segment_2_bottom_right[1])/2
    radius = (segment_1_top_left[1] - offset - (segment_2_bottom_right[1] - offset))/2
    circ_midpoint_x = (offset) + radius

    rr, cc = draw.rectangle(start = segment_2_top_left, end = (circ_midpoint_x,80+segment_1_top_left[1] - offset), shape = test_scene.shape)
    test_scene[rr.astype(int),cc.astype(int)] = 1 * trench_multiplier

    rr, cc = draw.disk(center = (circ_midpoint_x, circ_midpoint_y), radius = radius, shape = test_scene.shape)
    rr_semi = rr[rr < (circ_midpoint_x + 1)]
    cc_semi = cc[rr < (circ_midpoint_x + 1)]
    test_scene[rr_semi,cc_semi] = background_multiplier
    test_scene += scenes[np.random.randint(len(scenes))][0] * cell_multiplier
    test_scene = test_scene[segment_2_top_left[0]:segment_1_bottom_right[0],segment_2_top_left[1]:segment_1_bottom_right[1]]

    expanded_scene = np.zeros((int(test_scene.shape[0]*1.2), test_scene.shape[1]*2)) + trench_multiplier
    expanded_scene[expanded_scene.shape[0] - test_scene.shape[0]:,int(test_scene.shape[1]/2):int(test_scene.shape[1]/2) + test_scene.shape[1]] = test_scene
    return expanded_scene

plt.figure(figsize=(10,10))
plt.imshow(generate_PC_OPL(30,1,10))

In [ ]:
import gc
gc.collect()

In [ ]:
#PC_OPLs = np.array([generate_PC_OPL(*k) for k in grid])
#viewer = napari.view_image(np.array(PC_OPLs), rgb=False)

In [ ]:
real_phase = tifffile.imread("/home/georgeos/Storage/Dropbox (Cambridge University)/PhD_Georgeos_Hardo/ML_based_segmentation_results/40x_Ph2_test_1.5/top_trenches_PC/trench_{}/T_{}.tif".format(
    str(np.random.randint(56)).zfill(2),
    str(np.random.randint(20,25)).zfill(3)
))
plt.imshow(real_phase)

In [ ]:
def gaussian_2D(size, σ):
    x = np.linspace(0,size,size)
    μ = np.mean(x)
    A = 1/(σ*np.sqrt(2*np.pi))
    B = np.exp(-1/2 * (x-μ)**2/(σ**2))
    _gaussian_1D = A*B
    _gaussian_2D = np.outer(_gaussian_1D,_gaussian_1D)
    return _gaussian_2D

In [ ]:
condensers = {
    "Ph1": (0.45, 3.75, 24),
    "Ph2": (0.8, 5.0, 24),
    "Ph3": (1.0, 9.5, 24),
    "Ph4": (1.5, 14.0, 24),
    "PhF": (1.5, 19.0, 25)
} #W, R, Diameter

from scipy.special import jv, jve


def somb(x):
    z = np.zeros(x.shape)
    x = np.abs(x)
    idx = np.nonzero(x)
    z[idx] = 2*jv(1,np.pi*x[idx])/(np.pi*x[idx])
    return z

def get_phase_contrast_kernel(R,W,radius,scale,F,sigma):
    scale1 = 1000 # micron per millimeter
    F = F * scale1 # to microm
    Lambda = 0.55 # in micron % wavelength of light
    R = R * scale1 # to microm
    W = W * scale1 # to microm
    #The corresponding point spread kernel function for the negative phase contrast 

    meshgrid_arrange = np.arange(-radius,radius + 1,1)
    [xx,yy] = np.meshgrid(meshgrid_arrange,meshgrid_arrange)
    rr = np.sqrt(xx**2 + yy**2)*scale
    rr_dl = rr*(1/F)*(1/Lambda); # scaling with F and Lambda for dimension correction
    kernel1 = np.pi*R**2*somb(2*R*rr_dl);     
    kernel2 = np.pi*(R-W)**2*somb(2*(R-W)*rr_dl)


    kernel = kernel1 - 0*kernel2
    kernel = kernel/np.max(kernel)
    kernel[radius,radius] = kernel[radius,radius] + 1
    kernel = -kernel/np.sum(kernel)
    gaussian = gaussian_2D(radius*2+1, sigma)
    kernel = kernel * gaussian
    return kernel


W, R, diameter = condensers["Ph2"]
scale = 0.108379937 / 3 #0.35 #micron per pixel
min_sigma = 0.42*0.6/2 / scale # micron
kernel= get_phase_contrast_kernel(R, W, 50, scale, 5, 15)
scalebar = ScaleBar(scale, 'um')
plt.figure(figsize=(5.1,5.1))
plt.gca().add_artist(scalebar)
plt.imshow(kernel,cmap="Greys_r")
plt.axis("off")
plt.title("Phase Contrast")
plt.show()
kernel = cp.array(kernel)
kernel_np = kernel.get()
plt.figure(figsize = (15,10))
plt.plot(np.arange(-50,51)*scale,kernel_np[50,:])
plt.show()

In [ ]:
from skimage.util import random_noise
output = cuconvolve(cp.array(PC_OPLs[2000]),kernel)
output = output.get()
output = rescale(output, 1/4, anti_aliasing=False)[3:182,3:-3]
output = random_noise(output, mode="gaussian", mean=5,var=0.0000051,clip=False)
#viewer = napari.view_image(output_rescaled, rgb=False)

In [ ]:
top_segment = output
plt.figure(figsize=(5,5))
plt.imshow(top_segment,cmap="Greys_r")
top_segment.shape += (1,)
plt.show()

In [ ]:
def return_intersection_between_image_hists(img1, img2, bins):
    hist_1, _ = np.histogram(img1.flatten()/img1.flatten().max(), bins=bins)
    hist_2, _ = np.histogram(img2.flatten()/img2.flatten().max(), bins=bins)
    minima = np.minimum(hist_1, hist_2)
    intersection = np.true_divide(np.sum(minima), np.sum(hist_2))
    return intersection

In [ ]:
def convolve_rescale(image,kernel):
    output = cuconvolve(cp.array(image),cp.array(kernel))
    output = output.get()
    output = rescale(output, 1/4, anti_aliasing=False)
    return output

In [ ]:
def objective_function(z, ret_tuple = False):
    real_image = tifffile.imread("/home/georgeos/Storage/Dropbox (Cambridge University)/PhD_Georgeos_Hardo/ML_based_segmentation_results/40x_Ph2_test_1.5/top_trenches_PC/trench_{}/T_{}.tif".format(
    str(np.random.randint(1,56)).zfill(2),
    str(np.random.randint(20,25)).zfill(3)
))
    real_image = real_image.astype(np.float64)/np.max(real_image)
    σ, trench_multiplier, cell_multiplier, background_multiplier = z
    expanded_scene = generate_PC_OPL(trench_multiplier,cell_multiplier,background_multiplier)
    kernel = get_phase_contrast_kernel(R, W, 50, scale, 5, σ)
    convolved_image = convolve_rescale(expanded_scene,kernel)[0:182,3:-3]
    #convolved_image = random_noise(convolved_image, mode="gaussian", mean=5,var=σ2,clip=False)
    convolved_image = match_histograms(convolved_image, real_image, multichannel=False)
    convolved_image = resize(convolved_image,real_image.shape,clip=False,preserve_range=False,anti_aliasing=None)
    convolved_image = convolved_image/np.max(convolved_image)
    ssim_real = ssim(convolved_image, real_image)
    intersection = return_intersection_between_image_hists(convolved_image, real_image, 100)
    #sims 
    convolved_image.shape += (1,)
    real_image.shape += (1,)
    _fsim = fsim(convolved_image,real_image)
    _issm = issm(convolved_image,real_image)
    _sam = sam(convolved_image,real_image)
    _sre = sre(convolved_image,real_image)
    objs = [ssim_real, 0.5*intersection, _fsim, _issm, _sam, _sre/20]
    if ret_tuple == False:
        return -np.linalg.norm(objs)
    else:
        return objs

progress = []
def callbackF(x, f, context):
    print(x)
    progress.append(x)
    
def callbackSHGo(x):
    print(x)
    progress.append(x)

In [ ]:
bounds = list(zip([min_sigma, 1.0, 0.2, 1], [20.0, 30.0, 5.0, 20.0]))

In [ ]:
ret = dual_annealing(objective_function, bounds = np.array(bounds),callback=callbackF,maxiter=2000,initial_temp=20_000)

/home/georgeos/anaconda3/envs/SYMPTOMM2/lib/python3.9/site-packages/image_similarity_measures/quality_metrics.py:146: RuntimeWarning: divide by zero encountered in log2
  return -np.sum(np.nan_to_num(H * np.log2(H)))
/home/georgeos/anaconda3/envs/SYMPTOMM2/lib/python3.9/site-packages/image_similarity_measures/quality_metrics.py:146: RuntimeWarning: invalid value encountered in multiply
  return -np.sum(np.nan_to_num(H * np.log2(H)))
/home/georgeos/anaconda3/envs/SYMPTOMM2/lib/python3.9/site-packages/image_similarity_measures/quality_metrics.py:163: RuntimeWarning: invalid value encountered in double_scalars
  return numerator / denominator


[18.63509355 27.7073374   2.03421523  9.89497572]
[13.14532097 29.41646345  0.4122192   6.87297971]
[ 6.72465147 28.53609031  0.28009339 16.59469877]
[12.97900153 18.57782773  0.21842709  8.47863442]
[ 9.04359654 22.05011936  0.39259567 10.76147176]
[16.71266096 23.82911869  0.32420684  8.27122514]
[19.67243691 23.82911869  0.37869158  1.27339869]


In [ ]:
ret = shgo(objective_function, bounds = np.array(bounds),callback=callbackSHGo,options={"minimize_every_iter":True})

In [879]:
from skopt import gp_minimize
from skopt.callbacks import VerboseCallback
res = gp_minimize(objective_function,                  # the function to minimize
                  bounds,      # the bounds on each dimension of x
                  acq_func="gp_hedge",      # the acquisition function
                  n_calls=50,         # the number of evaluations of f
                  n_random_starts=5,  # the number of random initialization points
                 n_jobs = 1)   

/home/georgeos/anaconda3/envs/SYMPTOMM2/lib/python3.9/site-packages/image_similarity_measures/quality_metrics.py:146: RuntimeWarning: divide by zero encountered in log2
  return -np.sum(np.nan_to_num(H * np.log2(H)))
/home/georgeos/anaconda3/envs/SYMPTOMM2/lib/python3.9/site-packages/image_similarity_measures/quality_metrics.py:146: RuntimeWarning: invalid value encountered in multiply
  return -np.sum(np.nan_to_num(H * np.log2(H)))
/home/georgeos/anaconda3/envs/SYMPTOMM2/lib/python3.9/site-packages/image_similarity_measures/quality_metrics.py:163: RuntimeWarning: invalid value encountered in double_scalars
  return numerator / denominator
/home/georgeos/anaconda3/envs/SYMPTOMM2/lib/python3.9/site-packages/image_similarity_measures/quality_metrics.py:146: RuntimeWarning: divide by zero encountered in log2
  return -np.sum(np.nan_to_num(H * np.log2(H)))
/home/georgeos/anaconda3/envs/SYMPTOMM2/lib/python3.9/site-packages/image_similarity_measures/quality_metrics.py:146: RuntimeWarning: i

In [880]:
res.x

[19.649283784393557, 22.534062748590962, 0.2, 1.0]

In [980]:
def objective_function_get_image(z):
    σ, trench_multiplier, cell_multiplier, background_multiplier, = z
    expanded_scene = generate_PC_OPL(trench_multiplier,cell_multiplier,background_multiplier)
    kernel = get_phase_contrast_kernel(R, W, 50, scale, 5, σ)
    convolved_image = convolve_rescale(expanded_scene,kernel)[0:182,10:-10]
    #convolved_image = random_noise(convolved_image, mode="gaussian", mean=5,var=σ2,clip=False)
    convolved_image = match_histograms(convolved_image, real_image, multichannel=False)
    convolved_image = resize(convolved_image,real_image.shape,clip=False,preserve_range=False,anti_aliasing=None)
    convolved_image = convolved_image/np.max(convolved_image)
    ssim_real = ssim(convolved_image, real_image)
    intersection = return_intersection_between_image_hists(convolved_image, real_image, 100)
    return convolved_image

In [985]:
real_image = tifffile.imread("/home/georgeos/Storage/Dropbox (Cambridge University)/PhD_Georgeos_Hardo/ML_based_segmentation_results/40x_Ph2_test_1.5/top_trenches_PC/trench_{}/T_{}.tif".format(
    str(np.random.randint(1,56)).zfill(2),
    str(np.random.randint(1,25)).zfill(3)
))
optimised_image = objective_function_get_image(ret.x)
noisy_image = random_noise(optimised_image, mode="gaussian", mean=5,var=0.0002,clip=False)
viewer = napari.view_image(noisy_image, rgb=False)



viewer = napari.view_image(real_image, rgb=False)

<ipython-input-980-e5d310daa42f>:10: UserWarning: Inputs have mismatched dtype.  Setting data_range based on im1.dtype.
  ssim_real = ssim(convolved_image, real_image)


In [856]:
real_image.shape

(211, 40, 1)

In [151]:
convolved_outputs = []
for x in range(len(PC_OPLs)):
    output = cuconvolve(cp.array(PC_OPLs[x]),kernel)
    output = output.get()
    output = rescale(output, 1/1.75, anti_aliasing=False)[:364,:]
    output = random_noise(output, mode="gaussian", mean=5,var=0.85,clip=False)
    output_rescaled = match_histograms(output, real_phase[15][30:,:], multichannel=False)
    convolved_outputs.append(output_rescaled)
convolved_outputs = np.array(convolved_outputs)

In [153]:
viewer = napari.view_image(convolved_outputs**2, rgb=False)

In [ ]:
    scalebar = ScaleBar(scale, 'um')
    plt.figure(figsize=(5,5))
    plt.gca().add_artist(scalebar)
    plt.imshow(output_rescaled,cmap="Greys_r")
    plt.axis("off")
    plt.title("Phase Contrast")
    plt.show()

364

In [103]:
output.shape

(182, 97)

In [104]:
cuconvolve(cp.array(PC_OPLs[x]),kernel).shape

(912, 170)